.distance,
.geometry.boundary,.geometry.length
.apply
sql语句
schools["nearest_subway_distance"] = schools.geometry.apply(
    lambda school: subways.distance(school).min()
)
.overlay

In [ ]:
import geopandas as gpd

In [ ]:
schools = gpd.read_file("data/nyc/SchoolPoints_APS_2024_08_28 (1)/SchoolPoints_APS_2024_08_28.shp")
subways = gpd.read_file("data/nyc/nyc_subway_entrances/nyc_subway_entrances.shp")
bike_paths = gpd.read_file("data/nyc/New York City Bike Routes_20241223.geojson")
neighborhoods = gpd.read_file("data/nyc/custom-pedia-cities-nyc-Mar2018.geojson")
parks = gpd.read_file("data/nyc/Parks Properties_20241223.geojson")

In [ ]:
schools.explore()

In [ ]:
subways.explore()

In [ ]:
schools = schools.to_crs("EPSG:3857")
subways = subways.to_crs("EPSG:3857")
bike_paths = bike_paths.to_crs("EPSG:3857")
neighborhoods = neighborhoods.to_crs("EPSG:3857")
parks = parks.to_crs("EPSG:3857")

In [ ]:
import pandas as pd

In [ ]:
pd.options.display.float_format = '{:.4f}'.format

In [ ]:
# 1. GeoSeries.area
neighborhoods["area"] = neighborhoods.geometry.area
neighborhoods[['neighborhood', 'area']]

In [ ]:
neighborhoods.explore()

In [ ]:
parks.info(verbose=True)

In [ ]:
parks.explore()

In [ ]:
# 2. GeoSeries.boundary
parks["boundary"] = parks.geometry.boundary
parks_mapped = parks['boundary'].to_crs('EPSG:4326')
parks_mapped.explore()

In [ ]:
parks_mapped.geometry.bounds

In [ ]:
parks_mapped.geometry.total_bounds

In [ ]:
paths_2263 = bike_paths.to_crs('EPSG:2263')

In [ ]:
paths_2263.crs

In [ ]:
bike_paths["length"] = bike_paths.geometry.length
bike_paths[['segmentid', 'length']].sort_values(by=['length'], ascending=False)

In [ ]:
paths_2263["length"] = paths_2263.geometry.length
paths_2263[['segmentid', 'length']].sort_values(by=['length'], ascending=False)

In [ ]:
neighborhoods[neighborhoods['neighborhood'] == 'Chelsea']

In [ ]:
bike_paths[
    bike_paths['segmentid'].astype(str) == '299972.0'
    ].to_crs(
        'EPSG:4326'
        ).explore()

In [ ]:
oneSchool = schools.head(1)

In [ ]:
subways.distance(oneSchool).min()

In [ ]:
subways.distance(oneSchool).max()

In [ ]:
schools

In [ ]:
schools["max_subway_distance"] = schools.geometry.apply(
    lambda school: subways.distance(school).max()
)

In [ ]:
schools["nearest_subway_distance"] = schools.geometry.apply(
    lambda school: subways.distance(school).min()
)

In [ ]:
schools[['Name', 'nearest_subway_distance','max_subway_distance']].sort_values(by=['nearest_subway_distance'], ascending=True)

In [ ]:
neighborhoods.geometry.centroid.explore()

In [ ]:
bike_paths.geometry.buffer(10)


In [ ]:
bike_paths.head(100).geometry.buffer(100).explore()

In [ ]:
parks["intersects_bike_path"] = parks.geometry.apply(
    lambda park: bike_paths.geometry.intersects(park).any()
)

In [ ]:
parks.head(5)

In [ ]:
parks[parks['intersects_bike_path'] == True].explore()

In [ ]:
parks.geometry.minimum_bounding_circle().explore()

In [ ]:
neighborhoods.geometry.convex_hull.explore()

In [ ]:
neighborhoods.geometry.simplify(tolerance=0.001).explore()

In [ ]:
parks.geometry.envelope.explore()

In [ ]:
parks_neighborhoods_overlay = neighborhoods.overlay(parks, how="intersection")

In [ ]:
parks.head(2)

In [ ]:
parks_neighborhoods_overlay.head(2)

In [ ]:
print(len(parks_neighborhoods_overlay))
print(len(neighborhoods))

In [ ]:
parks_neighborhoods_overlay.explore()

In [ ]:
subways["translated"] = subways.geometry.translate(xoff=1000, yoff=1000)
subways.translated.explore()

In [ ]:
parks.geometry.is_valid

In [ ]:
neighborhoods.geometry.exterior.explore()

In [ ]:
bike_paths.geometry.geom_type


In [ ]:
schools["x"] = schools.geometry.to_crs(epsg=4326).x
schools["y"] = schools.geometry.to_crs(epsg=4326).y

In [ ]:
schools.info()

In [ ]:
schools[['x', 'Longitude', 'y', 'Latitude']]

In [ ]:
neighborhoods["contains_parks"] = neighborhoods.geometry.apply(
    lambda neighborhood: parks.geometry.apply(
        lambda park: neighborhood.contains(park)
    ).any()
)

In [ ]:
neighborhoods[neighborhoods['contains_parks'] == False].explore()

In [ ]:
bike_paths["crosses_parks"] = bike_paths.geometry.apply(
    lambda path: path.crosses(parks.geometry).any()
)

In [ ]:
bike_paths[bike_paths['crosses_parks'] == True].explore()

In [ ]:
parks["disjoint_neighborhoods"] = parks.geometry.apply(
    lambda park: park.disjoint(neighborhoods.geometry).all()
)

In [ ]:
parks[parks['disjoint_neighborhoods'] == True].explore()

In [ ]:
def schoolsWithin500(school):
    return school.distance(subways.geometry).any() <= 500

In [ ]:
schools["within_500m_subway"] = schools.geometry.apply(
    schoolsWithin500
)


In [ ]:
subway_union = subways.geometry.union_all()
schools["within_500m_subway"] = schools.geometry.distance(subway_union) <= 500


In [ ]:
schools[schools["within_500m_subway"] == True].explore()

In [ ]:
parks["overlaps_neighborhoods"] = parks.geometry.apply(
    lambda park: park.overlaps(neighborhoods.geometry).any()
)


In [ ]:
parks[parks["overlaps_neighborhoods"] == True].explore()

In [ ]:
parks['geometry'].symmetric_difference(neighborhoods['geometry']).explore()